In [1]:
import pandas as pd

In [35]:
# 2014 
df = pd.read_csv(r'L:\T2040\soundcast_2014\outputs\daysim\_trip.tsv', sep='\t')
hh = pd.read_csv(r'L:\T2040\soundcast_2014\outputs\daysim\_household.tsv', sep='\t')

In [36]:
# AAA Calculations availavble for small, medium, large Sedans (and average of these), SUV, Minivan

# Analysis of survey could be used to allocate vehicle type distributions?

# http://publicaffairsresources.aaa.biz/wp-content/uploads/2016/03/2016-YDC-Brochure.pdf
# results are averages for sedans

# Costs by miles
# Operating costs total (gas + maintenance + tries)
depreciation_per_mile = 3759.0/15000    # depreciation by mile

# Annual costs
insurance = 1222    #(full-coverage insurance, license, registration, taxes, depreciation @ 15,000 miles per year), finance charge
taxes = 687    # license, registration, taxes
finance = 683    # load financing

annual_fixed_costs = insurance+taxes+finance

In [37]:
# get total travel cost (based on vmt)
# includes 
# Does NOT include parking
hh_mile_costs = df.groupby('hhno').sum()[['travcost']]
hh_mile_costs = hh_mile_costs.reset_index()

hh = pd.merge(hh, hh_mile_costs, on='hhno', how='left')

# Some households have 0 travel cost
hh['travcost'] = hh['travcost'].fillna(0)

In [38]:
# Total vehicle miles traveled per household to calculate depreciation
driver_trips = df[df['dorp'] == 1]

In [39]:
hh_travdist = df.groupby('hhno').sum()[['travdist']]
hh_travdist = hh_travdist.reset_index()

In [40]:
hh = pd.merge(hh, hh_travdist, on='hhno', how='left')

In [41]:
# hh['annual_vehicle_costs'] = hh

# Weekday travel

# Annualalize costs as 262
annual_factor = 262

# annual operating costs
hh['operating_cost_drivers'] = hh['travcost']*annual_factor
hh['depreciation'] = hh['travdist']*depreciation_per_mile*annual_factor

# fixed costs as function of number of vehicles owned
hh['veh_insurance'] = hh['hhvehs']*insurance
hh['veh_taxes'] = hh['hhvehs']*taxes
hh['veh_finance'] = hh['hhvehs']*finance

hh['annual_auto_costs'] = hh['operating_cost_drivers']+hh['depreciation']+hh['veh_insurance']+hh['veh_taxes']+hh['veh_finance']
# Need to add per mile depreciation costs

In [42]:
# Calculate transit cost
transit_trips = df[df['mode'] == 6]

hh_travdist_transit_cost = transit_trips.groupby('hhno').sum()[['travcost']]
hh_travdist_transit_cost['transit_cost'] = hh_travdist_transit_cost['travcost']
hh_travdist_transit_cost = hh_travdist_transit_cost.reset_index()
hh_travdist_transit_cost['annual_transit_cost'] = hh_travdist_transit_cost['transit_cost']*annual_factor

In [43]:
hh = pd.merge(hh, hh_travdist_transit_cost[['hhno','annual_transit_cost']], on='hhno', how='left')

In [45]:
hh['annual_transit_cost'] = hh['annual_transit_cost'].fillna(0)

In [ ]:
# Parking Costs?

In [46]:
hh.to_csv(r'hh.csv',index=False)

In [128]:
# Add GPS coordinates to parcel file
parcel = pd.read_csv(r'L:\T2040\soundcast_2014\inputs\accessibility\parcels_urbansim.txt', delim_whitespace=True)